In [ ]:
from tqdm.notebook import tqdm
from math import sqrt,erf
from multiprocessing import Pool
from itertools import product, repeat
from matplotlib import pyplot as plt
from matplotlib import animation
import matplotlib as mpl
import numpy as np
from varname import nameof
from qutip import *
from matplotlib.cm import get_cmap
import colorcet as cc
import subprocess
import os, shutil
from more_itertools import intersperse
import time
import functools
from scipy import special
from IPython.display import display, clear_output


plt.style.use('ggplot')
markers = ["o", "X", "P", "p", "*"]
cols = [p['color'] for p in plt.rcParams['axes.prop_cycle']]
plt.rc('text.latex', preamble=r'\usepackage{amsmath}\usepackage{braket}\usepackage{nicefrac}')
plt.rcParams.update({'font.size': 30,
                     'figure.figsize': (11,7),
                     'axes.facecolor': 'white',
                     'axes.edgecolor': 'lightgray',
                     "figure.autolayout": 'True',
                     'axes.xmargin': 0.03,
                     'axes.ymargin': 0.05,
                     'axes.grid': False,
                     'axes.linewidth': 5,
                     'lines.markersize': 15,
                     'text.usetex': True,
                     'lines.linewidth': 8,
                     "legend.frameon": True,
                     "legend.framealpha": 0.7,
                     "legend.handletextpad": 1,
                     "legend.edgecolor": "black",
                     "legend.handlelength": 1,
                     "legend.labelspacing": 0,
                     "legend.columnspacing": 1,
                     "legend.fontsize": 35,
                    })
linestyles = ["-", "--", ":"]
bbox = dict(boxstyle="round", facecolor="white")

deltaD = 0.1
plt.plot([], [])
plt.show()
clear_output()

In [ ]:
def get_RG_flow(D0, J0, Ub_by_J=0, plot=False):
    """ Returns the flow of couplings in the form of two ndarrays J and D.
    Each ndarray is in ascending order of the bandwidth. """
    
    Ub = -Ub_by_J * J0
    omega = -D0/2
    
    ### initialise arrays with UV values
    D = [D0]
    J = [J0]
    
    ### apply URG transformations until bandwith vanishes
    ### or J reduces to zero.
    while D[-1] >= deltaD and J[-1] >= 0:
        
        ### URG equation
        deltaJ = - J[-1] * (J[-1] + 4 * Ub) / (omega - D[-1]/2 + J[-1]/4) * deltaD
        
        ### Check if denominator has changed sign, 
        ### then append renormalised values to array
        if (omega - (D[-1] - deltaD)/2 + (J[-1] + deltaJ)/4) * (omega - D0/2 + J0/4) > 0:
            D.append(D[-1] - deltaD)
            J.append(J[-1] + deltaJ)
        else:
            break
    
    ### plot values
    if plot:
        plt.plot(np.array(D)/D0, np.array(J)/J0, marker="o")
        plt.xlabel(r"$D/D_0$")
        plt.ylabel(r"$J/J_0$")
    return np.flip(D), np.flip(J)

_ = get_RG_flow(2, 0.3, Ub_by_J=0, plot=True)

In [ ]:
def get_renorm(new_coefficients, savepath, total_num, alpha):
    """ Calculates and returns the renormalised coefficients at a given RG step. """

    q_pos = total_num - 2 ### position of the nearest IOM
    try:
        ### uncomment the next line if you don't want to read from saved data
        raise ValueError('A very bad thing happened.')
        
        ### read from saved data, if available
        new_coefficients = np.load(savepath)['arr_0']
    except:
        ### if saved data is not available, run this block
        
        ### pad earlier coefficients with zeros to make room for new IOM
        enlarged_coeffs = np.ravel([[0, 0, 0, c] for c in new_coefficients])
        new_coefficients = np.copy(enlarged_coeffs)
        
        ### loop over spins of IOM, particle/hole sectors as well as cloud members
        for beta, p_h, k_pos in product([1, -1], [1, 0], range(1, q_pos-1, 2)):
            beta_pos = int((1 - beta) / 2)
            betabar_pos = int(1 - beta_pos)

            ### generate all possible occupancies for qbeta, kbeta, kbetabar and impurity
            ### these are generated only when required and not stored in memory, so as to
            ### reduce physical memory usage.
            occ = np.tile(np.repeat([1, 0], 2**(total_num - 1 - q_pos - beta_pos)), 2**(q_pos + beta_pos))
            delta_q = occ == 1 - p_h
            not_delta_q = occ == p_h
            
            occ = np.tile(np.repeat([1, 0], 2**(total_num - 1 - k_pos - beta_pos)), 2**(k_pos + beta_pos))
            delta_kbeta = occ == p_h
            not_delta_kbeta = occ == 1 - p_h

            occ = np.tile(np.repeat([1, 0], 2**(total_num - 1 - k_pos - betabar_pos)), 2**(k_pos + betabar_pos))
            delta_kbetabar = occ == p_h
            not_delta_kbetabar = occ == 1 - p_h

            
            occ = np.repeat([1, 0], 2**(total_num - 1))
            delta_imp = occ == abs((1 - beta)/2 - p_h)
            not_delta_imp = occ == 1 - abs((1 - beta)/2 - p_h)


            ### simply apply the analytical formulae
            new_coefficients[(delta_kbeta) & (delta_q)] += alpha * beta * ((occ - 0.5) * enlarged_coeffs)[(not_delta_kbeta) & (not_delta_q)]
            new_coefficients[(delta_kbetabar) & (delta_q) & (delta_imp)] += alpha * enlarged_coeffs[(not_delta_kbetabar) & (not_delta_q) & (not_delta_imp)]
                    
        ### normalise the final coefficients
        new_coefficients /= np.linalg.norm(new_coefficients)
        
        ### Save (with compression!)
        np.savez_compressed(savepath, new_coefficients)
    return new_coefficients


def init_wavefunction(num_in):
    """ Generates the initial wavefunction at the fixed point.
    Returns the coefficients associated with it and the list
    of all possible occupancies for each of the members.
    No IOMS are taken into account at this point."""
    
    total_num = 1 + 2 * num_in
    
    ### Generate the set of all possible combinations [C1, C2, ...]
    ### where C1=11111...1, C2=1111..10 and so on.
    combinations = np.array(list(product([1,0], repeat=total_num)))
    
    ### Generate a set of lists [O1, O2, ...], where Oi refers to the 
    ### ith member and is itself an array. This array Oi is of the form 
    ### Oi = [occ(C1,i), occ(C2,i), ...], where occ(Cj,i) 
    ### is the occupancy of the ith member in the configuration Cj.
    occupancies = [combinations[:,i] for i in range(total_num)]
    
    ### Now generate the coefficients corresponding
    ### to the initial singlet state.
    coefficients = np.zeros(2**total_num)
    
    ### loop over all momenta and spins of the Kondo
    ### cloud and pick out those that create the singlet.
    for k_pos in range(1, 2 * num_in, 2):
        for k_spin in [1, -1]:
            d_spin = - k_spin
            combination = np.zeros(total_num)
            combination[0] = (1 + d_spin)/2
            combination[int(k_pos + (1 - k_spin)/2)] = 1
            coefficients[(combinations == combination).all(axis=1)] = d_spin
            
    return np.array(coefficients)/np.linalg.norm(coefficients), occupancies


def get_tensorRG(D0, J0, num_in, num_out):
    omega = -D0/2
    D, J = get_RG_flow(D0, J0)
    
    ### ensure that there are enugh RG steps in J and D
    ### to recouple all IOMS.
    assert len(J) >= num_out
    
    ### obtain the initial coefficients and set of occupancies
    coefficients, occupancies_init = init_wavefunction(num_in)
    occupancies = np.copy(occupancies_init)
    
    ### save the initial set of coefficients and store the savepath
    ### in the list of savepaths that will be finally returned.
    coefficients_arr = [coefficients]
    savepaths = []
    savepath = "./MERGCoeffs/{:.0f},{:.10f},{:.10f},{:.10f},{:.10f}.npz".format(1 + 2 * num_in, J0, D0, deltaD, omega)
    savepaths.append(savepath)
    os.makedirs("./MERGCoeffs", exist_ok=True)
    np.savez_compressed(savepath, coefficients)
    
    ### initial the next set of coefficients and loop through 
    ### the RG flow to generate the tensor network RG.
    new_coefficients = coefficients
    for Ji, Di in tqdm(zip(J[:num_out], D[:num_out]), total=num_out):
        total_num = 1 + 2 * num_in + 2
        alpha = Ji / (omega - Di/2 + Ji/4)
        savepath = "./MERGCoeffs/{:.0f},{:.10f},{:.10f},{:.10f},{:.10f}.npz".format(total_num, Ji, Di, deltaD, omega)
        
        ### get the next set of coefficients and append the savepath to access them later.
        new_coefficients = get_renorm(new_coefficients, savepath, total_num, alpha)
        savepaths.append(savepath)
        
        ### increase the number of entangled members.
        num_in += 1
        
    return savepaths, occupancies_init


def get_VNE_perstep(args):
    """Returns the VNE at a given RG step."""
    
    savepath, occupancy = args
    
    ### Access the coefficients data for the given RG step.
    coefficients = np.load(savepath)['arr_0']
    
    ### Create the reduced density matrix by adding 
    ### the pertinent coefficients.
    rho_reduced = np.zeros([2,2])
    for occ_0, occ_1 in product([0,1],repeat=2):
        rho_reduced[occ_0][occ_1] = np.sum(coefficients[occupancy==occ_0]*coefficients[occupancy==occ_1])

    ### Diagonalise reduced DM and return VNE.
    eigvals = np.linalg.eigvalsh(rho_reduced)
    return -np.sum(eigvals * np.log(eigvals))


def get_VNE_RG(D0, J0, num_in, num_out, member):
    """Function to obtain the RG flow of the entanglement entropy
    of a particular member"""
    
    ### Get the list of savepaths for the coefficients along the RG flow
    savepaths, occupancies_init = get_tensorRG(D0, J0, num_in, num_out)

    ### Generate the occupancies of the member, 
    ### by repeating appropriate number of times.
    occupancy = [occupancies_init[member]]
    for run in range(1, len(savepaths)):
        occupancy.append(np.repeat(occupancy[-1], 4))
        
    ### Calculate VNE from the coefficients data.
    ### Pool() applies multiprocessing.
    args = [[savepath, occupancy[run]] for run, savepath in enumerate(savepaths)]
    vne_arr = list(tqdm(Pool().imap(get_VNE_perstep, args), total=len(args)))
 
    plt.plot(range(num_out+1), vne_arr, marker='o')
    plt.xlabel(r"RG steps")
    plt.ylabel(r"S$_\text{EE}$")
    plt.show()
    return vne_arr

In [ ]:
vne_arr = get_VNE_RG(2, 0.2, 1, 4, 1)

In [ ]:
def bruteForceMERG(D0, J0, num_in, num_out):
    omega = -D0/2
    D, J = get_RG_flow(D0, J0)
    assert len(J) >= num_out
    total_dim = 2 + 2 * (num_in + num_out)
    c_all = [tensor([sigmaz()]*(i) + [destroy(2)] + [identity(2)]*(total_dim - i -1)) for i in range(total_dim)]
    Sdz = 0.5 * (c_all[0].dag() * c_all[0] - c_all[1].dag() * c_all[1])
    vacuum = tensor([basis(2,0)]*total_dim)
    init_state = 0
    for k_pos in range(2, 2 * num_in + 1, 2):
        init_state += c_all[0].dag() * c_all[k_pos + 1].dag() * vacuum
        init_state += - c_all[1].dag() * c_all[k_pos].dag() * vacuum
    gstates = [init_state.unit()]
    ref_state1 = c_all[7].dag() * c_all[3] * gstates[0]


    for Ji, Eq in tqdm(zip(J[:num_out], D[:num_out]), total=num_out):
        q_pos = 2 + 2 * num_in
        alpha = Ji / (omega - Eq/2 + Ji/4)
        eta = 0
        eta_dag = 0
        for k_pos in range(2, q_pos-1, 2):
            for beta in [1, -1]:
                beta_pos = 0 if beta == 1 else 1
                eta += alpha * Sdz * beta * c_all[k_pos + beta_pos].dag() * c_all[q_pos + beta_pos]
                eta += alpha * c_all[beta_pos].dag() * c_all[1 - beta_pos] * c_all[k_pos + 1 - beta_pos].dag() * c_all[q_pos + beta_pos]
                eta_dag += alpha * Sdz * beta * c_all[q_pos + beta_pos].dag() * c_all[k_pos + beta_pos]
                eta_dag += alpha * c_all[1 - beta_pos].dag() * c_all[beta_pos] * c_all[q_pos + beta_pos].dag() * c_all[k_pos + 1 - beta_pos]
        gstates.append(((1 + eta + eta_dag) * gstates[-1]).unit())
        num_in += 1
        num_out -= 1

    for i,state in enumerate(gstates):
        rho_k = (state * state.dag()).ptrace([2])
        print ((ref_state1.dag() * state)[0][0][0], entropy_vn(rho_k))
        plt.scatter(i, entropy_vn(rho_k))
        
bruteForceMERG(2, 0.2, 1, 4)

In [ ]:
total_num = 3
np.array(list(product([1,0], repeat=total_num)))[:,1]